In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, StackingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV,KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv('data/hw7/train.csv')

In [3]:
valid=pd.read_csv('data/hw7/validation.csv')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7595 entries, 0 to 7594
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 7595 non-null   int64  
 1   Customer_Age              7595 non-null   int64  
 2   Gender                    7595 non-null   object 
 3   Dependent_count           7595 non-null   int64  
 4   Education_Level           7595 non-null   object 
 5   Marital_Status            7595 non-null   object 
 6   Income_Category           7595 non-null   object 
 7   Card_Category             7595 non-null   object 
 8   Months_on_book            7595 non-null   int64  
 9   Total_Relationship_Count  7595 non-null   int64  
 10  Months_Inactive_12_mon    7595 non-null   int64  
 11  Contacts_Count_12_mon     7595 non-null   int64  
 12  Credit_Limit              7595 non-null   float64
 13  Total_Revolving_Bal       7595 non-null   int64  
 14  Avg_Open

In [5]:
valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2532 entries, 0 to 2531
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 2532 non-null   int64  
 1   Customer_Age              2532 non-null   int64  
 2   Gender                    2532 non-null   object 
 3   Dependent_count           2532 non-null   int64  
 4   Education_Level           2532 non-null   object 
 5   Marital_Status            2532 non-null   object 
 6   Income_Category           2532 non-null   object 
 7   Card_Category             2532 non-null   object 
 8   Months_on_book            2532 non-null   int64  
 9   Total_Relationship_Count  2532 non-null   int64  
 10  Months_Inactive_12_mon    2532 non-null   int64  
 11  Contacts_Count_12_mon     2532 non-null   int64  
 12  Credit_Limit              2532 non-null   float64
 13  Total_Revolving_Bal       2532 non-null   int64  
 14  Avg_Open

In [6]:
data.drop(['CLIENTNUM'], axis=1).duplicated().sum()

0

In [7]:
x_train=data.drop(['CLIENTNUM','Attrition_Flag'],axis=1)
y_train=data['Attrition_Flag']

In [8]:
def data_prepare(data):
    obj_feature=list(data.describe(include='object').columns)
    labeled=pd.get_dummies(data[obj_feature], prefix=obj_feature)
    data_labeled=data.join(labeled).drop(obj_feature, axis=1)
    data_labeled=data_labeled.drop(['CLIENTNUM','Attrition_Flag_Existing Customer','Gender_F'],axis=1)
    return data_labeled
    

In [9]:
data_labeled=data_prepare(data)

In [10]:
data_labeled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7595 entries, 0 to 7594
Data columns (total 37 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Customer_Age                      7595 non-null   int64  
 1   Dependent_count                   7595 non-null   int64  
 2   Months_on_book                    7595 non-null   int64  
 3   Total_Relationship_Count          7595 non-null   int64  
 4   Months_Inactive_12_mon            7595 non-null   int64  
 5   Contacts_Count_12_mon             7595 non-null   int64  
 6   Credit_Limit                      7595 non-null   float64
 7   Total_Revolving_Bal               7595 non-null   int64  
 8   Avg_Open_To_Buy                   7595 non-null   float64
 9   Total_Amt_Chng_Q4_Q1              7595 non-null   float64
 10  Total_Trans_Amt                   7595 non-null   int64  
 11  Total_Trans_Ct                    7595 non-null   int64  
 12  Total_

In [11]:
valid['Attrition_Flag']='Existing Customer'#создаим исскуственно чтобы засунуть в функцию, использовать не будем.
valid_labeled=data_prepare(valid)

In [12]:
X_valid=valid_labeled

In [13]:
X=data_labeled.drop(['Attrition_Flag_Attrited Customer'],axis=1)
y=data_labeled['Attrition_Flag_Attrited Customer']


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13,stratify=y)

In [15]:
scaler = MinMaxScaler()
X = scaler.fit_transform(X)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

In [16]:
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve, auc,
                             classification_report)

In [17]:
learning_rate = 0.1
n_estimators = 400
max_depth_in = 10
random_state_in = 13
max_features=20

In [18]:
from sklearn.metrics import classification_report

Сделаем Бустинг из sclearn

In [19]:
reg = GradientBoostingClassifier(n_estimators = n_estimators, max_depth=max_depth_in, learning_rate = learning_rate, random_state=random_state_in)
reg.fit(X_train, y_train)
y_pred_xgb = reg.predict(X_test)
#y_valid_xgb= reg.predict(X_valid)
print(accuracy_score(y_test, y_pred_xgb))
print(f1_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb, digits=4))


0.9637919684002634
0.8812095032397408
              precision    recall  f1-score   support

           0     0.9692    0.9882    0.9786      1275
           1     0.9315    0.8361    0.8812       244

    accuracy                         0.9638      1519
   macro avg     0.9504    0.9122    0.9299      1519
weighted avg     0.9632    0.9638    0.9630      1519



Попробуем RandomForest

In [20]:
rf=RandomForestClassifier(n_estimators=n_estimators,  random_state=random_state_in, max_features=max_features)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

print(accuracy_score(y_test, y_pred_rf))
print(f1_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf, digits=4))


0.9637919684002634
0.881720430107527
              precision    recall  f1-score   support

           0     0.9700    0.9875    0.9786      1275
           1     0.9276    0.8402    0.8817       244

    accuracy                         0.9638      1519
   macro avg     0.9488    0.9138    0.9302      1519
weighted avg     0.9632    0.9638    0.9631      1519



In [21]:
import xgboost
from xgboost import XGBClassifier

/home/danil/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/home/danil/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/danil/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(


Попробуем бустинг из библиотки xgboost. Тут много экспериментировали с параметрами, выбрали лучшие. 

In [22]:
estimator = XGBClassifier()

parameters = {
    "eta":[0.2,0.3,0.31],
#    "gamma": [0],
#    "lambda": [0.5,1,2],
#     "min_samples_leaf": [0.01,0.05],
    "max_depth":[3,6,7],
#    "max_features":[10,15,20],
#    "criterion": ["friedman_mse"],
    "subsample":[ 1.0],
    "n_estimators":[200],
    'n_jobs':[10,20]
    }



skf=KFold(n_splits=3)
clf = GridSearchCV(estimator=estimator, scoring='f1',param_grid=parameters,n_jobs = 10, cv=skf,verbose=True)

clf.fit(X, y)
param=clf.best_params_
print(clf.best_params_)

Fitting 3 folds for each of 18 candidates, totalling 54 fits


/home/danil/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/home/danil/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/danil/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_cla

[14:05:32] WARNING: ./src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:05:32] WARNING: ./src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:05:32] WARNING: ./src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/danil/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/home/danil/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/danil/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
/home/danil/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.16.0-unknown is an invalid version and will not be supported in a future release
  warnings.warn(
/home/danil/.local/lib/python3.10/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be sup

[14:05:33] WARNING: ./src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:05:33] WARNING: ./src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[14:05:33] WARNING: ./src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[14:05:56] WARNING: ./src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/lib/python3/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:05:56] WARNING: ./src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/lib/python3/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[14:05:59] WARNING: ./src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
{'eta': 0.3, 'max_depth': 3, 'n_estimators': 200, 'n_jobs': 10, 'subsample': 1.0}


In [23]:
model = XGBClassifier(**param)
#eta=0.26, gamma=0,n_estimators=200
model.fit(X_train, y_train)
y_pred_gb=model.predict(X_test)


print(accuracy_score(y_test, y_pred_gb))
print(f1_score(y_test, y_pred_gb))
print(classification_report(y_test, y_pred_gb, digits=4))

[14:06:00] WARNING: ./src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.9697169190256748
0.9029535864978903
              precision    recall  f1-score   support

           0     0.9767    0.9875    0.9821      1275
           1     0.9304    0.8770    0.9030       244

    accuracy                         0.9697      1519
   macro avg     0.9536    0.9323    0.9425      1519
weighted avg     0.9693    0.9697    0.9694      1519



XGBClassifier  дал лучшие результаты на тестовой выборке им и воспользуемся.

In [24]:
model.fit(X, y)
y_valid_gb=model.predict(X_valid)

[14:06:00] WARNING: ./src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/usr/lib/python3/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [25]:
valid['churn']=y_valid_gb

In [26]:
valid[['CLIENTNUM','churn']].to_csv('data/hw7/7_result_bobrovmv.csv', index=False)